In [1]:
from os import listdir
from os.path import isfile, join

import pandas as pd
import numpy as np
from skimage import io, transform

import xml.etree.ElementTree as ET
import xmltodict
import cv2 
import csv

import matplotlib.patches as patches

In [2]:
train_classifier_dir = "./data/train_classifier/"
csv_path = "./data/train_classifier/train_list.csv"
classes = ['aquarium', 'bottle', 'bowl', 'box', 'bucket', 'plastic_bag', 'plate', 'styrofoam', 'tire', 'toilet', 'tub', 'washing_machine', 'water_tower']

def get_rect_axis(bndbox):
    xmin = int(bndbox["xmin"])
    xmax = int(bndbox["xmax"])
    ymin = int(bndbox["ymin"])
    ymax = int(bndbox["ymax"])
    return xmin, ymin, xmax-xmin, ymax-ymin

def show_img(img_path, xml_data):
    image = io.imread(img_path)
    fig, ax = plt.subplots(1)
    ax.imshow(image)
    
    if type(xml_data["object"]) == list:
        for i, ob in enumerate(xml_data["object"]):
            x, y , width, height = get_rect_axis(ob["bndbox"])
            
            rect = patches.Rectangle((x, y), width, height, linewidth=1, edgecolor='r', facecolor='none')
            ax.add_patch(rect)
    else:
        x, y , width, height = get_rect_axis(xml_data["object"]["bndbox"])
        
        rect = patches.Rectangle((x, y), width, height, linewidth=1, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
        
    plt.show()
    
def xml_parser(xml_path):
    xml_data = None
    with open(xml_path) as fd:
        xml_data = xmltodict.parse(fd.read())
    return xml_data["annotation"]
       
    
def csv_writer(csvfilename, filename, label):
    with open(csvfilename, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow([filename, label])

In [3]:
def data_process(img_path, xml_data):
    image = io.imread(img_path)
    if type(xml_data["object"]) == list:
        for i, ob in enumerate(xml_data["object"]):
            x, y , width, height = get_rect_axis(ob["bndbox"])
            
            filename = xml_data["filename"][:-4] + "_" + str(i) +  xml_data["filename"][-4:]
            crop_img = image[y:y+height, x:x+width]
            cv2.imwrite(train_classifier_dir + filename, crop_img)
            
            
            csv_writer(csv_path, filename, classes.index(ob["name"]))
    else:
        x, y , width, height = get_rect_axis(xml_data["object"]["bndbox"])
        
        crop_img = image[y:y+height, x:x+width]
        cv2.imwrite(train_classifier_dir + xml_data["filename"], crop_img)
        csv_writer(csv_path, xml_data["filename"], classes.index(xml_data["object"]["name"]))

In [7]:
train_image_dir = "./data/train_cdc/train_images/"
train_annotation_dir = "./data/train_cdc/train_annotations/"

train_image_path = [join(train_image_dir, f) for f in listdir(train_image_dir) if isfile(join(train_image_dir, f))]
train_annotation_path = [join(train_annotation_dir, f) for f in listdir(train_annotation_dir) if isfile(join(train_annotation_dir, f))]

train_image_path.sort()
train_annotation_path.sort()

for i, a in zip(train_image_path, train_annotation_path):
    xml_data = xml_parser(a)
#     show_img(i, xml_data)
    if xml_data.get("object") is not None:
        data_process(i, xml_data)

In [8]:
import PIL.Image

In [11]:
size = 64, 64

for i, f in enumerate(listdir(train_classifier_dir)):
    if isfile(join(train_classifier_dir, f)) and f[-3:] != 'csv':
        filename = join(train_classifier_dir, f)
        try:
            im = PIL.Image.open(filename)
            im = im.resize(size, PIL.Image.ANTIALIAS)
            im.save(filename, "JPEG")
        except IOError:
            print("cannot create thumbnail for %s" % filename)